In [39]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('./train.csv', index_col=False, sep=',', header=0)
df.head()

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,4,2,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,6,9,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,5,6,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,8,5,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,3,5,2,2,6,6,4,1,2,3,5,0.415


## Data cleaning

In [3]:
def prerpocessing(df):
    df.drop('id', axis=1, inplace=True)

df_cleaned = df.copy()
prerpocessing(df_cleaned)

## Training

In [4]:
X = df_cleaned.drop('FloodProbability', axis=1)
y = df_cleaned['FloodProbability']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [48]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation=None))

In [49]:
def r2_score(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

ann.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=[r2_score])

In [50]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 25)

Epoch 1/25
24456/24456 [==============================] - 15s 590us/step - loss: 7.3169e-04 - r2_score: 0.6974
Epoch 2/25
24456/24456 [==============================] - 14s 592us/step - loss: 3.7328e-04 - r2_score: 0.8427
Epoch 3/25
24456/24456 [==============================] - 14s 587us/step - loss: 3.6690e-04 - r2_score: 0.8449
Epoch 4/25
24456/24456 [==============================] - 14s 589us/step - loss: 3.6485e-04 - r2_score: 0.8458
Epoch 5/25
24456/24456 [==============================] - 14s 591us/step - loss: 3.6357e-04 - r2_score: 0.8464
Epoch 6/25
24456/24456 [==============================] - 15s 595us/step - loss: 3.6264e-04 - r2_score: 0.8469
Epoch 7/25
24456/24456 [==============================] - 14s 592us/step - loss: 3.6233e-04 - r2_score: 0.8471
Epoch 8/25
24456/24456 [==============================] - 15s 595us/step - loss: 3.6181e-04 - r2_score: 0.8472
Epoch 9/25
24456/24456 [==============================] - 14s 593us/step - loss: 3.6191e-04 - r2_score: 0.8470
E

In [51]:
from sklearn.metrics import r2_score

train_preds = ann.predict(X_train)
test_preds = ann.predict(X_test)

print(f"Train R2 score: {r2_score(y_train, train_preds)}")
print(f"Test R2 score: {r2_score(y_test, test_preds)}")

10481/10481 [==============================] - 3s 242us/step
Train R2 score: 0.8634117195793833
Test R2 score: 0.864244745065786


## Create submission

In [52]:
test_df = pd.read_csv('./test.csv', header=0, sep=',')
test_df.head()

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,1117957,4,6,3,5,6,7,8,7,8,4,8,5,7,5,6,3,6,4,4,5
1,1117958,4,4,2,9,5,5,4,7,5,4,2,4,7,4,5,1,7,4,4,3
2,1117959,1,3,6,5,7,2,4,6,4,2,7,9,2,5,5,2,3,6,8,3
3,1117960,2,4,4,6,4,5,4,3,4,4,7,8,4,6,7,6,4,2,4,4
4,1117961,6,3,2,4,6,4,5,5,3,7,4,3,2,6,4,6,8,4,5,5


In [53]:
test_ids = test_df['id']
prerpocessing(test_df)

In [54]:
X_to_pred = sc.transform(test_df)

In [55]:
preds = ann.predict(X_to_pred)

23291/23291 [==============================] - 6s 250us/step


In [56]:
preds = [p[0] for p in preds]

This reached .8631 on Kaggle

In [57]:
submission_df = pd.DataFrame([test_ids.values, preds]).T
submission_df.columns = ['id', 'FloodProbability']

In [58]:
submission_df.to_csv('./submission.csv', index=False)